<a href="https://colab.research.google.com/github/ertgrulyksk/Weekday-illegal-public-transportation-analysis/blob/main/Weekday_illegal_public_transportation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd 

In [3]:
!pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot as plt
from matplotlib import style
import seaborn as sns
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import pandas_profiling
import altair as alt

import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv("Border_Crossing_Entry_Data.csv")
data.head()

,Port Name,State,Port Code,Border,Date,Measure,Value
0,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Personal Vehicle Passengers,1414
1,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Personal Vehicles,763
2,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Truck Containers Empty,412
3,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Truck Containers Full,122
4,Alcan,AK,3104,US-Canada Border,2/1/2020 00:00,Trucks,545


In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355511 entries, 0 to 355510
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Port Name  355511 non-null  object
 1   State      355511 non-null  object
 2   Port Code  355511 non-null  int64 
 3   Border     355511 non-null  object
 4   Date       355511 non-null  object
 5   Measure    355511 non-null  object
 6   Value      355511 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 19.0+ MB
None


In [7]:
data['Date'] = pd.to_datetime(data['Date'])   # convert objecg to datetime

In [8]:
data.columns = data.columns.str.lower()  # change columns name to lower case to make it easy:

In [9]:
# reset dataframe: add year & day of the week for further analysis
data['year'] = data['date'].dt.strftime('%Y')
data['day_of_week'] = data['date'].dt.day_name()

data.head(3)

,port name,state,port code,border,date,measure,value,year,day_of_week
0,Alcan,AK,3104,US-Canada Border,2020-02-01,Personal Vehicle Passengers,1414,2020,Saturday
1,Alcan,AK,3104,US-Canada Border,2020-02-01,Personal Vehicles,763,2020,Saturday
2,Alcan,AK,3104,US-Canada Border,2020-02-01,Truck Containers Empty,412,2020,Saturday


In [10]:
# The busiest border
busy_border = data.groupby("border").value.sum().reset_index()
busy_border.sort_values(["value"], axis=0, 
                 ascending=[False], inplace=True)  
busy_border

,border,value
1,US-Mexico Border,7337300710
0,US-Canada Border,2776127401


In [11]:
# Global treemap breakdown
fig = px.treemap(data, path=['border','state','measure'], values='value',
                  color='value', hover_data=['state'],
                  color_continuous_scale='RdBu')

fig.update_layout(title="US inbound crossing with Canada and Mexico: State level",
                  width=780, height=600, uniformtext=dict(minsize=10, mode='hide'))  

fig.show()

ZeroDivisionError: ignored

In [12]:
# The transportation way of inbound crossing for each US border
# visualization with Seaborn

border_tick=range(2)

fig2, ax2 = plt.subplots(figsize=(18,8))

ax2 = sns.barplot(x=data['border'], y=data['value'], hue=data['measure'], data=data, log=True)
ax2.set_xlabel('US borders', size=14, fontweight='bold')
ax2.set_ylabel('Inbound crossing value', size=14, fontweight='bold')
plt.legend(loc="upper left", frameon=False, ncol=3, fontsize=12)
plt.title("US inbound crossing analysis: Measure VS Border", size=17, color="indianred", fontweight='bold')

# bar chart styling
sns.despine(left=True)  # remove upper and left line

ax2.set_axisbelow(True)  
ax2.yaxis.grid(True, color='#EEEEEE')
ax2.xaxis.grid(False)

plt.show()

In [13]:
# Now, let's look at state level yearly transportation evolution

tran = data.groupby(['measure', 'state','year']).value.sum().reset_index()

fig = px.scatter(tran, x="year", y="value", color="measure", facet_col="state",
       facet_col_wrap=5)

fig.update_layout(title="US inbound crossing measure from Canada and Mexico border 1996-2020", 
                  width=780, height=600)

fig.update_xaxes(showgrid=False)

fig.show()

In [14]:
# Use pivot table for day of week analysis 

# regroup related data
week_table = data.groupby(['border', 'day_of_week']).value.count().reset_index()

# Create pivot table
pivot_week = pd.pivot_table(data, 'value', ['border','state','year'], 'day_of_week')

# Reorder column
pivot_week = pivot_week[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday']]
pivot_week.head()

day_of_week                       Monday      Tuesday    Wednesday  \
border           state year                                          
US-Canada Border AK    1996  1505.222222   342.277778  1339.972222   
                       1997   877.708333  2216.513889   291.500000   
                       1998  3564.250000  1018.236111  2775.152778   
                       1999   278.555556  3657.861111  1069.736111   
                       2000   648.777778  1972.833333   342.472222   

day_of_week                     Thursday       Friday     Saturday  \
border           state year                                          
US-Canada Border AK    1996  1676.750000   298.111111  3011.000000   
                       1997  1409.750000  3136.027778   290.935185   
                       1998   326.666667  1610.166667  3526.222222   
                       1999  2217.763889   337.194444  1707.333333   
                       2000  3644.333333  1078.888889  1702.787037   

day_of_week                       Sunday  
border           state year               
US-Canada Border AK    1996   835.208333  
                       1997  3084.916667  
                       1998   292.398148  
                       1999  3065.250000  
                       2000   399.750000

In [28]:
sns.set()
pd.pivot_table(data, 'value', ['state'], 'day_of_week').plot(kind= 'bar', width=0.9, alpha=0.9, figsize=(22, 10), title="US inbound crossing weekday level", fontsize=15)
plt.ylabel("crossing number", fontsize=18, color='indianred', fontweight='bold')
plt.xlabel("US states", fontsize=18, color='indianred', fontweight='bold')
plt.title('US inbound crossing weekday level', fontsize=25, color='indianred', fontweight='bold')
plt.legend(fancybox=True, shadow=True, fontsize=16)

In [16]:
# create new column which will return "illegal" if matches certain condition for measure.
data['law'] = data.measure.apply(lambda x: 'illegal' if x == 'Truck Containers Empty' or x == 'Truck Containers Full' or x =='Rail Containers Empty' or x == 'Rail Containers Full' else 'legal')

# Then, extract all categorical data: illegal crossing
illegal_data = data[data.law == 'illegal']
illegal_data2 = illegal_data.groupby(['year', 'border']).value.sum().reset_index()
illegal_data2.head()

,year,border,value
0,1996,US-Canada Border,2109673
1,1996,US-Mexico Border,3354140
2,1997,US-Canada Border,2968205
3,1997,US-Mexico Border,3634946
4,1998,US-Canada Border,6120191


In [17]:
fig = px.area(illegal_data2, x="year", y="value", color="border")

fig.update_layout(title="US illegal inbound crossing from Canada and Mexico border 1996-2020", 
                  width=780, height=500)

fig.update_xaxes(showgrid=False)

fig.show()

In [18]:
# legal VS illegal crossing analysis: state level

measure_ill = data[data.law == 'illegal']
measure_ill = measure_ill.groupby(['border', 'measure', 'state', 'year']).value.sum().reset_index()
measure_ill.head()

# set pivot table: columns should be illega crossing measure
pivot_illegal = pd.pivot_table(measure_ill, 'value', ['state'], 'measure')
pivot_illegal.head(5)

measure,Rail Containers Empty,Rail Containers Full,Truck Containers Empty,Truck Containers Full
state,,,,
AK,0.00,0.00,2990.12,6840.44
AZ,22892.48,30611.52,84950.68,262494.44
CA,9672.24,1196.00,411093.48,660817.44
ID,5844.60,77539.64,8623.08,48217.84
ME,21067.16,18944.24,95059.60,250444.04


In [19]:
# plot pivot table outcome with Seaborn

sns.set()
pd.pivot_table(measure_ill, 'value', ['state'], 'measure').plot(kind= 'bar', width=0.88, figsize=(22, 10), title="US inbound crossing weekday level", fontsize=15)
plt.ylabel("crossing number", fontsize=18, color='indianred', fontweight='bold')
plt.xlabel("US states", fontsize=18, color='indianred', fontweight='bold')
plt.title('US inbound crossing illegal transportation', fontsize=25, color='indianred', fontweight='bold')
plt.legend(fancybox=True, shadow=True, fontsize=16)

In [20]:
# Extract legal dataset
all_legal = data[data.law == 'legal']

# Regroup and rename measure elements in dataset
all_legal.loc[(all_legal['measure'] == 'Bus Passengers') | (all_legal['measure'] == 'Train Passengers') | (all_legal['measure'] == 'Buses') | (all_legal['measure'] == 'Trains'), 'measure'] = 'public'
all_legal.loc[(all_legal['measure'] == 'Personal Vehicle Passengers') | (all_legal['measure'] == 'Personal Vehicles') | (all_legal['measure'] == 'Pedestrians') | (all_legal['measure'] == 'Trucks'), 'measure'] = 'private'
all_legal.head()

,port name,state,port code,border,date,measure,value,year,day_of_week,law
0,Alcan,AK,3104,US-Canada Border,2020-02-01,private,1414,2020,Saturday,legal
1,Alcan,AK,3104,US-Canada Border,2020-02-01,private,763,2020,Saturday,legal
4,Alcan,AK,3104,US-Canada Border,2020-02-01,private,545,2020,Saturday,legal
5,Alexandria Bay,NY,708,US-Canada Border,2020-02-01,public,1174,2020,Saturday,legal
6,Alexandria Bay,NY,708,US-Canada Border,2020-02-01,public,36,2020,Saturday,legal


In [21]:
# create final sorted table: pu_pri
pu_pri = all_legal[['year', 'state', 'measure', 'value']]

# create pivot table show private & public
pivot_pupri = pd.pivot_table(pu_pri, 'value', ['state','year'], 'measure')

pivot_pupri = pivot_pupri.reset_index()   # reconvert pivot table back to dataframe

# Calculate percentage of public 
pivot_pupri['per_public'] = pivot_pupri.apply(lambda row: row['public']/(row['private']+row['public']), axis=1)
pivot_pupri.head()

measure,state,year,private,public,per_public
0,AK,1996,2749.437500,957.826389,0.258365
1,AK,1997,2664.812500,1144.166667,0.300387
2,AK,1998,3054.486111,1322.645833,0.302172
3,AK,1999,2710.680556,1352.777778,0.332913
4,AK,2000,2730.597222,1349.375000,0.330731


In [22]:
# Now, visualize public measure percentage

fig = px.scatter(pivot_pupri, x="year", y="per_public", facet_col="state",
       facet_col_wrap=5)

fig.update_layout(title="public transportation percentage from 1966 to 2020", 
                  width=780, height=600)

fig.layout.yaxis2.update(matches=None)  # matches=none, reset y value
fig.update_xaxes(showgrid=False)
fig.show()

In [23]:
# Explore Alaska data
alaska = data.groupby(['year', 'measure', 'state']).value.sum().reset_index()
alaska.head()

alaska = alaska[alaska.state == 'AK'].reset_index()
alaska.head()

ooo = alaska[(alaska.year == '2017') | (alaska.year == '2018') | (alaska.year == '2019')]
nnn = ooo[(ooo.measure == 'Bus Passengers') | (ooo.measure == 'Train Passengers') | (ooo.measure == 'Buses') | (ooo.measure == 'Trains')]  
nnn.head(12)

,index,year,measure,state,value
252,3528,2017,Bus Passengers,AK,188239
253,3542,2017,Buses,AK,12220
257,3622,2017,Train Passengers,AK,93387
258,3635,2017,Trains,AK,311
262,3690,2018,Bus Passengers,AK,210359
263,3704,2018,Buses,AK,11893
267,3785,2018,Train Passengers,AK,102946
268,3798,2018,Trains,AK,322
272,3853,2019,Bus Passengers,AK,217102
273,3867,2019,Buses,AK,14375


In [24]:
# Create pivot table

AK = pd.pivot_table(nnn, 'value', ['year', 'state'], 'measure').reset_index()
AK.head()

measure,year,state,Bus Passengers,Buses,Train Passengers,Trains
0,2017,AK,188239,12220,93387,311
1,2018,AK,210359,11893,102946,322
2,2019,AK,217102,14375,103563,270
